# CHAPTER 1: INTRODUCING THE EXECUTION PLAN
An execution plan is a set of instructions for executing a query. Devised by the SQL Server Query Optimizer, an execution plan describes the set of operations that the execution engine needs to perform to return the data required by a query.

The execution plan is your window into the SQL Server query optimizer and query execution engine. It will reveal which tables and indexes a query accessed, in which order, how they were accessed, what types of joins were used, how much data was retrieved initially, and at what point filtering and sorting occurred. It will show how aggregations were performed, how calculated columns were derived, how and where foreign keys were accessed, and more.

Any problems created by the query will frequently be apparent within the execution plan, making it an excellent tool for troubleshooting poorly-performing queries. Rather than guess at why a query is sending your I/O through the roof, you can examine its execution plan to identify the exact operation, and associated section of T-SQL code, that is causing the problem. For example, the plan may reveal that a query is reading every row in a table or index, even though only a small percentage of those rows are being used in the query. By modifying the code within the WHERE clause, SQL Server may be able to devise a new plan that uses an index to find directly (or seek) only the required rows.

This chapter will introduce execution plans. We'll explore the basics of obtaining an execution plan and start the process of learning how to read them, covering the following topics:

* **A brief background on the query optimizer** – execution plans are a result of the optimizer's operations, so it's useful to know at least a little bit about what the optimizer does, and how it works.
* **The plan cache and plan reuse** – execution plans are usually stored in an area of memory called the plan cache and may be reused. We'll discuss why plan reuse is important.
* **Actual and estimated execution plans** – clearing up the confusion over estimated versus actual execution plans and how they differ.
* **Capturing an execution plan** – we'll capture a plan for a simple query and introduce some of the basic elements of a plan, and the information they contain.

## What Happens When a Query is Submitted?

Every time we submit a query to SQL Server, several server processes kick into action; their job collectively is to manage the querying or modification of that data. Within the relational engine, the query is parsed by the Parser, bound by the Algebrizer and then finally optimized by the Query Optimizer, where the most important part of the work occurs. Collectively, we refer to these processes as query compilation. The SQL Server relational engine takes the input, which is the SQL text of the submitted query, and compiles it into a plan to execute that query. In other words, the process generates an execution plan, effectively a series of instructions for processing the query. 
 
![Query Compilation and Execution](Graphics/Figure1-1.jpg)

*Figure 1-1: Query compilation and execution*

The plan generated is stored in an area of memory called the plan cache. The next time the optimizer sees the same query text, it will check to see if a plan for that SQL text exists in the plan cache. If it does, it will pass the cached plan on to the query execution engine, bypassing the full optimization process.
The query execution engine will execute the query, according to the instructions laid out in the execution plan. It will generate calls to the storage engine, the process that manages access to disk and memory within SQL Server, to retrieve and manipulate data as required by the plan.

### Query compilation phase
Since execution plans are created and managed from within the relational engine, that's where we'll focus our attention in this book. The following sections review briefly what happens during query compilation, covering the parsing, binding, and particularly the optimization phase, of query processing. 
### Query parsing
When a request to execute a T-SQL query reaches SQL Server, either an ad hoc query from a command line or application program, or a query in a stored procedure, user-defined function, or trigger, the query compilation and execution process can begin, and the action starts in the rel¬ational engine. 

As the T-SQL arrives in the relational engine, it passes through a process that checks that the T-SQL is written correctly, that it's well formed. This process is query parsing. If a query fails to parse correctly, for example, if you type SELETC instead of SELECT, then parsing stops and SQL Server returns an error to the query source. The output of the Parser process is a parse tree, or query tree (or it's even called a sequence tree). The parse tree represents the logical steps necessary to execute the requested query.
### Query binding
If the T-SQL string has parsed correctly, the parse tree passes to the algebrizer, which performs a process called query binding. The algebrizer resolves all the names of the various objects, tables, and columns referred to within the query string. It identifies, at the individual column level, all the data types (varchar(50) versus datetime and so on) for the objects being accessed. It also determines the location of aggregates, such as SUM and MAX, within the query, a process called aggregate binding. 

This algebrizer process is important because the query may have aliases or synonyms, names that don't exist in the database, that need to be resolved, or the query may refer to objects not in the database. When objects don't exist in the database, SQL Server returns an error from this step, defining the invalid object name (except in the case of deferred name resolution). As an example, the algebrizer would quickly find the table Person.Person in the AdventureWorks database. However, the Product.Person table, which doesn't exist, would cause an error and the whole compilation process would stop. 
### Stored procedure and deferred name resolution
On creating a stored procedure, its statement text is parsed and stored in sys.sql_modules catalog view. However, the tables referenced by the text do not have to exist in the database at this point. This gives more flexibility because, for example, the text can reference a temporary table that is not created by the stored procedure, and does not yet exist, but that we know will exist at execution time. At execution time, the query processor finds the names of the objects referenced, in sys.sql_modules, and makes sure they exist.

The algebrizer outputs a binary called the query processor tree, which is then passed on to the query optimizer. The output also includes a hash, a coded value representing the query. The optimizer uses the hash to determine whether there is already a plan for this query stored in the plan cache, and whether the plan is still valid. A plan is no longer considered valid after some changes to the table (such as adding or dropping indexes), or when the statistics used in the optimization were refreshed since the plan was created and stored. If there is a valid cached plan, then the process stops here and the cached plan is reused.
## Query optimization
The query optimizer is a piece of software that considers many alternate ways to achieve the requested query result, as defined by the query processor tree passed to it by the algebrizer. The optimizer estimates a "cost" for each possible alternative way of achieving the same result, and attempts to find a plan that is cheap enough, within as little time as is reasonable.

Most queries submitted to SQL Server will be subject to a full cost-based optimization process, resulting in a cost-based plan. Some very simple queries can take a "fast track" and receive what is known as a trivial plan.

### Full cost-based optimization

The full cost-based optimization process takes three inputs:

* __The query processor tree__ – gives the optimizer knowledge of the logical query structure and of the underlying tables and indexes.
* **Statistics** – index and column statistics give the optimizer an understanding of volume and distribution of data in the underlying data structures.
* __Constraints__ – the primary keys, enforced and trusted referential constraints and any other types of constraints in place on the tables and columns that make up the query tell the optimizer the limits on possible data stored within the tables referenced.

Using these inputs, the optimizer applies its model, essentially a set of rules, to transform the logical query tree into a plan containing a set of operators that, collectively, will physically execute the query. Each operator performs a dedicated task. The optimizer uses various operators for accessing indexes, performing joins, aggregations, sorts, calculations, and so on. For example, the optimizer has a set of operators for implementing logical join conditions in the submitted query. It has one specialized operator for a Nested Loops implementation, one for a Hash Match, one for a Merge, and one for an Adaptive Join.

The optimizer will generate and evaluate many possible plans, for each candidate testing different methods of accessing data, attempting different types of join, rearranging the join order, trying different indexes, and so on. Generally, the optimizer will choose the plan that its calculations suggest will have the lowest total cost, in terms of the sum of the estimated CPU and I/O processing costs. 
During these calculations, the optimizer assigns a number to each of the steps within the plan, representing its estimation of the combined amount of CPU and disk I/O time it thinks each step will take. This number is the estimated cost for that step. The accumulation of costs for each step is the estimated cost for the execution plan itself. We'll shortly cover the estimated costs, and why they are estimates, in more detail.

Plan evaluation is a heuristic process. The optimizer is not attempting to find the best possible plan but rather the lowest-cost plan in the fewest possible iterations, meaning the shortest amount of time. The only way for the optimizer to arrive at a perfect plan would be to be able to take an infinite amount of time. No one wants to wait that long on their queries.

Having selected the lowest-cost plan it could find within the allotted number of iterations, the query execution component will use this plan to execute the query and return the required data. As noted earlier, the optimizer will also store the plan in the plan cache. If we submit a subsequent request with identical SQL text, it will bypass the entire compilation process and simply submit the cached plan for execution. A parameterized query will be parsed, and if a plan with a matching query hash is found in the cache, the remainder of the process is short-circuited.

### Trivial plans
For very simple queries, the optimizer may simply decide to apply a trivial plan, rather than go through the full cost-based optimization process. The optimizer's rules for deciding when it can simply use a trivial plan are unclear, and probably complex. However, for example, a very simple query, such as a SELECT statement against a single table with no aggregates or calculations, as shown in Listing 1-1 would receive a trivial plan.

In [0]:
--Listing 1-1
SELECT d.Name
FROM HumanResources.Department AS d
WHERE d.DepartmentID = 42;

Adding even one more table, with a JOIN, would make the plan non-trivial. Also, if additional indexes exist on the table, or if the possibility of parallelism exists (discussed more in Chapter 13), then you will get further optimization of the plan. 

It's also worth noting here that this query falls within the rules covered by auto-parameterization, so the hard-coded value of "42" will be replaced with a parameter when the plan is stored in cache, to enable plan reuse. We'll cover that in more detail in Chapter X. 

All data manipulation language (DML) statements are optimized to some extent, even if they receive only a trivial plan. However, some types of data definition language (DDL) statement may not be optimized at all. For example, if a CREATE TABLE statement parses correctly, then there is only one "right way" for the SQL Server system to create a table. Other DDL statements, such as using ALTER TABLE to add a constraint, will go through the optimization process. 

## Query execution phase

The query execution engine executes the query per the instructions set out in the execution plan. At runtime, the execution engine cannot change the optimizer's plan. However, it can under certain circumstances force a plan to be recompiled. For example, if we submit to the query processor a batch or a stored procedure containing multiple statements, the whole batch will be compiled at once, with plans produced for every statement. Even if we have IF…THEN or CASE flow control in our queries, all statements within the batch will be compiled. At runtime, each plan is checked to ensure it's still valid. As for plans taken in the plan cache, if the plan's associated statement references tables that have changed, or had statistics updated, since the plan was compiled, then the plan is no longer considered valid. If that occurs, then the execution is temporarily halted, the compilation process is invoked, and the optimizer will produce a new plan, only for the affected statement in the batch or procedure.

Introduced in SQL Server 2017, there is also the possibility of interleaved execution when the object being referenced in the query is a multi-statement table-valued user-defined function. During an interleaved execution, the optimizer generates a plan for the query, in the usual fashion, then the optimization phase pauses, the pertinent subtree of a given plan is executed to get the actual row counts, and the optimizer then uses the actual row counts to optimize the remainder of the query. We'll cover interleaved execution and multi-statement table-valued user-defined functions in more detail in Chapter X.

## Working with the Optimizer

Most application developers, when writing application code, are used to exerting close control, not just over the required result of a piece of code, but also over how, step by step, that outcome should be achieved. Most compiled languages work in this manner. SQL Server and T-SQL behave in a different fashion.
The query optimizer, not the database developer, decides how a query should be executed. We focus solely on designing a T-SQL query to describe logically the required set of data. We do not, and should not, attempt to dictate to SQL Server how to execute it.

What this means in practice is the need to write efficient SQL, which generally means using a set-based approach that describes as succinctly as possible, in as few statements as possible, just the required data set. This is the topic for a whole other book, and one that's already been written by Itzik Ben-Gan, Inside SQL Server T-SQL Querying.

However, beyond that, there are some practical ways that the database developer or DBA can help the optimizer generate efficient plans, and avoid unnecessary plan generation:

* maintaining accurate, up-to-date statistics
* promoting plan reuse.

### The importance of statistics
As we've discussed, the optimizer will choose the lowest-cost plan, based on estimated cost. The principal driver of these estimates are the statistics on your indexes and data. Ultimately, this means that the quality of the plan choice is limited by the quality of the statistics the optimizer has available for the target tables and indexes.
We don't want the optimizer to read all the data in all the tables referenced in a query each time it tries to generate a plan. Instead, the optimizer relies on statistics, aggregated information based on a sample of the data, that provides the information used by the optimizer to represent the entire collection of data. 
The estimated cost of an execution plan depends largely on its cardinality estimations, in other words, its knowledge of how many rows are in a table, and its estimations of how many of those rows satisfy the various search and join conditions, and so on. 

> __New Cardinality Estimator in SQL Server 2014__

> In SQL Server 2014, the cardinality estimator within SQL Server was updated for the first time since SQL Server 7.0. It's very likely that you may see a difference in plans generated in SQL Server 2014 compared to previous versions, just because of the update to the cardinality estimator, let alone any updates to other processes within the optimizer.

These cardinality estimations rely on statistics collected on columns and indexes within the database that describe the data distribution, i.e. the number of different values present, and how many occurrences of each value. This in turn determines the selectivity of the data. If a column is unique, then it will have the highest possible selectivity, and the selectivity degrades as the level of uniqueness decreases. A column such as "gender," for example, will likely have a low selectivity.

If statistics exist for a relevant column or index, then the optimizer will use them in its calculations. If statistics don't exist then, by default, they'll be created immediately, in order for the optimizer to consume them.

The information that makes up statistics is divided into three subsections:

* the header – general data about a given set of statistics
* the density graph – the selectivity, uniqueness, of the data, and, most importantly
* a histogram – a tabulation of counts of the occurrence of a particular value, taken from up to 200 data points that are chosen to best represent the complete data in the table. 

It's this "data about the data" that provides the information necessary for the optimizer to make its calculations. The key measure is selectivity, i.e. the percentage of rows that pass the selection criteria. The worst possible selectivity is 1.0 (or 100%) meaning that every row will pass. The cardinality for a given operator in the plan is then simply the selectivity of that operator multiplied by the number of input rows.

The reliance the optimizer has on statistics means that your statistics need to be as accurate as possible, or the optimizer could make poor choices for the execution plans it creates. Statistics, by default, are created and updated automatically within the system for all indexes or for any column used as a predicate, as part of a WHERE clause or JOIN criteria. 

The automatic update of statistics that occurs, assuming it's on, only samples a subset of the data in order to reduce the cost of the operation. This means that, over time, the statistics can become a less-and-less-accurate reflection of the actual data. All of this can lead to SQL Server making poor choices of execution plans.

There are other statistical considerations too, around the objects types we choose to use in our SQL code. For example, table variables do not ever have statistics generated on them, so the optimizer makes assumptions about them, regardless of their actual size. Prior to SQL Server 2014, that assumption was for one row. SQL Server 2014 and SQL Server 2016 now assume one hundred rows in multi-statement user defined functions, but remain with the one row for all other objects. SQL Server 2017 can, in some instances, use interleaved execution to arrive at more accurate row counts for these functions.

Temporary tables do have statistics generated on them and their statistics are stored in the same type of histogram as permanent tables, and the optimizer can make use of these statistics. In places where statistics are needed, say, for example, when doing a JOIN to a temporary table, you may see advantages in using a temporary table over a table variable. However, further discussion of such topics is beyond the scope of this book.

As you can see from all the discussion about statistics, their creation and maintenance has a large impact on your systems. More importantly, statistics have a large impact on your execution plans. For more information on this topic, check out Erin Stellato's article Managing SQL Server Statistics in Simple-Talk (http://preview.tinyurl.com/yaae37gj).

## The plan cache and plan reuse

All the processes described previously, which are required to generate execution plans, have an associated CPU cost. For simple queries, SQL Server generates an execution plan in less than a millisecond, but for very complex queries, it can take seconds or even minutes to create an execution plan.

Therefore, SQL Server will store plans in a section of memory called the plan cache, and reuse those plans wherever possible, to reduce that overhead. Ideally, if the optimizer encounters a query it has seen before, it can bypass the full optimization process and just select the plan from the cache.

However, there are a few reasons why the plan for a previously executed query may no longer be in the cache. It may have been aged out of the cache to make way for new plans, or forced out due to memory pressure, or someone manually clearing the cache. In addition, certain changes to the underlying database schema, or statistics associated with these objects, can cause plans to be recompiled (i.e. recreated from scratch).

### Plan aging

Each plan has an associated "age" value that is the estimated CPU cost of compiling the plan multiplied by the number of times it has been used. So, for example, a plan with an estimated compilation cost of 10 that has been referenced 5 times has an "age" value of 50. The idea is that frequently-referenced plans that are expensive to compile will remain in the cache for as long as possible. Plans undergo a natural aging process. The lazywriter process, an internal process that works to free all types of cache (including the plan cache), periodically scans the objects in the cache and decreases this value by one each time. 

Plans will remain in the cache unless there is a specific reason they need to be moved out. For example, if the system is under memory pressure, plans may be aged, and cleared out, more aggressively. Also, plans with the lowest age value can be forced out of the cache if the cache is full and memory is required to store newer plans. This can become a problem if the optimizer is being forced to produce a very high volume of plans, many of which are only ever used one time by one query, constantly forcing older plans to be flushed from the cache. This a problem known as cache churn, which we'll discuss again shortly.

## Manually clearing the plan cache

Sometimes, during testing, you may want to flush all plans from the cache, to see how long a plan takes to compile, or to investigate how minor query adjustments might lead to slightly different plans. The command DBCC FREEPROCCACHE will clear the cache for all databases on the server. In a production environment, that can result in a significant and sustained performance hit because then each subsequent query is a "new" query and must go through the optimization process. We can flush only specific queries or plans by supplying a plan_handle or sql_handle. You can retrieve these values from either the plan cache itself using Dynamic Management Views (DMVs) such as sys.dm_exec_query_stats, or the Query Store (see Chapter X). Once you have the value, simply run DBCC FREEPROCCACHE(<plan_handle>) to remove a specific plan from the plan cache.

Similarly, we can use DBCC FLUSHPROCINDB(db_id) to remove all plans for a specific database, but the command is not officially documented. SQL Server 2016 introduced a new, fully-documented, method to remove all plans for a single database, which is to run the following command within the target database:


In [0]:
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;

### Criteria for plan reuse

When we submit a query to the server, the algebrizer process creates a hash value for the query. The optimizer stores the hash value in the QueryHash property of the associated execution plan (covered in more detail in Chapter 2). The job of the QueryHash is to identify queries with the same, or very similar logic (there are rare cases where logically different queries end up with the same hash value, known as hash collisions). 

For each submitted query, the optimizer looks for a matching QueryHash value among the plans in the plan cache. If found, it performs a detailed comparison of the SQL text of the submitted query and SQL text associated with the cached plan. If they match exactly (including spaces and carriage returns), this returns the plan_handle, a value that uniquely identifies the plan in memory. This plan may be reused, if the following are also true:

* __the plan was created using the same SET options__ (see Chapter 2) – otherwise there will be multiple plans created even if the SQL Texts are identical
* __the database ID's match__ – identical queries against different databases will have separate plans.

Note that it's also possible that lack of schema-qualification for the referenced objects in the query will lead to separate plans for different users.

Generally, however, a plan will be reused if all four of the above match (QueryHash, SQL Text, SET options, database ID). If so, the entire cost of the optimization process is skipped and the execution plan in the plan cache is reused.

### Avoiding cache churn: query parameterization

It is an important best practice to write queries in such a way that SQL Server can reuse the plans in cache. If we submit ad hoc queries to SQL Server and use hard-coded literal values then, for most of those queries, SQL Server will be forced to complete the full optimization process and compile a new plan each time. On a busy server, this can quickly lead to cache bloat, and to older plans being forced relatively quickly from the cache.

For example, let's say we submit the query in Listing 1-2

SELECT  p.ProductID ,
        p.Name AS ProductName ,
        pi.Shelf ,
        l.Name AS LocationName
FROM    Production.Product p
        INNER JOIN Production.ProductInventory AS pi
             ON pi.ProductID = p.ProductID
        INNER JOIN Production.Location AS l
             ON l.LocationID = pi.LocationID
WHERE   l.Name = 'Paint';



In [0]:
--Listing 1-2
SELECT p.ProductID,
       p.Name AS ProductName,
       pi.Shelf,
       l.Name AS LocationName
FROM Production.Product p
    INNER JOIN Production.ProductInventory AS pi
        ON pi.ProductID = p.ProductID
    INNER JOIN Production.Location AS l
        ON l.LocationID = pi.LocationID
WHERE l.Name = 'Paint';

We then submit the same query again, but for a different location name (say, "Tool Cribs" instead of "Paint"). This will result in two separate plans stored in cache, even though the two queries are essentially the same (they will have the same QueryHash values, assuming no other changes are made).

To ensure plan reuse, it's best to use either stored procedures or parameterized queries, where the variables within the query are identified with parameters, rather than hard-coded literals, and we simply pass in the required parameter values at runtime. This way, the SQL text the optimizer see will be "set in stone," maximizing the possibility of plan reuse.

These are also called "prepared queries" and are built from the application code. For an example of using prepared statements, see this article in Technet (http://preview.tinyurl.com/ybvc2vcs). You can also parameterize a query by using sp_executesql from within your T-SQL code. 
Another way to mitigate the churn from ad hoc queries is to use a server setting called "Optimize For Ad Hoc Workloads." Turning this on will cause the optimizer to create what is known as a "plan stub" in the plan cache, instead of putting the entire plan there the first time a plan is created. This means that single-use plans will take up radically less memory in your plan cache.

### Plan recompilation

Certain events and actions, such as changes to an index used by a query, can cause a plan to be recompiled, which simply means that the existing plan will be marked for recompilation, and a new plan generated the next time the query is called. It is important to remember this, because recompiling execution plans can be a very expensive operation. This only becomes a problem if our actions as programmers force SQL Server to perform excessive recompilations.

We'll discuss recompiles in more detail in Chapter X, but the following actions can lead to recompilation of an execution plan (see http://preview.tinyurl.com/y947r969 for a full list):
* changing the structure of a table, view or function referenced by the query
* changing, or dropping, an index used by the query 
* updating the statistics used by the query
* calling the function sp_recompile
* mixing DDL and DML within a single batch
* changing certain SET options within the T-SQL of the batch
* changes to cursor options within the query
* deferred compiles
* changes to a remote rowset if you're using a function like OPENQUERY.

# Getting Started with Execution Plans

Execution plans assist us in writing efficient T-SQL code, troubleshooting existing T-SQL behavior or monitoring and reporting on our systems. How we use them and view them is up to us, but first we need to understand what information is contained within the plans, and how to interpret that information. One of the best ways to learn about execution plans is to see them in action, so let's get started.

## Permissions required to view execution plans

In order to view execution plans for queries you must have the correct permissions within the database. If you are sysadmin, dbcreator or db_owner, you won't need any other permission. If you are granting this permission to developers who will not be in one of those privileged roles, they'll need to be granted the ShowPlan permission within the database being tested. Run the statement in Listing 1-3.



In [0]:
--Listing 1-3
GRANT SHOWPLAN TO [username];

Substituting the username will enable the user to view execution plans for that database. Additionally, in order to run the queries against the Dynamic Management Objects (DMO), either VIEW SERVER STATE or VIEW DATABASE STATE, depending on the DMO in question, will be required. We'll explore DMOs more in Chapter X. 
Execution plan formats

SQL Server can output the execution plan in three different ways:
* as an XML plan
* as a text plan
* as a graphical plan.

The one you choose will depend on the level of detail you want to see, and on the methods used to capture or view that plan.

In each format, we can retrieve the execution plan without executing the query, so without runtime information, which is known as the estimated plan, or retrieve the plan with added runtime information, which of course requires executing the query, and is known as the actual plan. While, strictly speaking, the terms actual and estimated are exclusive to graphical plans, it is common to see them applied to all execution plan formats and, for simplicity, we'll use those terms for each format here.

### XML plans

XML plans present a complete set of data available on a plan, all on display in the structured XML format. The XML format is great for transmitting to other data professionals if you want help on an execution plan or need to share with coworkers. Using XQuery, we can also query the XML data directly (see Chapter 8).
We can use one of the following two commands to retrieve the plan in XML format:
* __SET SHOWPLAN_XML ON__ – generates the estimated plan (i.e. the query is not executed).
* __SET STATISTICS_XML ON__ – generates the actual execution plan (i.e. with runtime information).

XML plans are extremely useful, but mainly for querying, not for standard style reading of plans since the XML is not human readable. Useful though these types of plan are, you're more likely to use graphical plans for simply browsing the execution plan.

Every graphical execution plan is actually XML under the covers. Within SSMS, simply right-click on the plan itself. From the context menu select Show Execution Plan XML… to open a window with the XML of the execution plan.

### Text plans

These can be quite difficult to read, but detailed information is immediately available. Their text format means that they we can copy or export them into text manipulation software such as NotePad or Word, and then run searches against them. While the detail they provide is immediately available, there is less detail overall from the execution plan output in these types of plan, so they can be less useful than the other plan types.

Text plans are on the deprecation list from Microsoft. They will not be available in a future version of SQL Server. I don't recommend using them.

Nevertheless, here are the possible commands we can use to retrieve the plan in text format:

* __SET SHOWPLAN_ALL ON__ – retrieves the estimated execution plan for the query.
* __SET STATISTICS PROFILE ON__ – retrieves the actual execution plan for the query.
* __SET SHOWPLAN_TEXT ON__ – retrieves the estimated plan but with a very limited set of data, for use with tools like osql.exe.

### Graphical plans

Graphical plans are the most commonly viewed format of execution plan. They are quick and easy to read. We can view both estimated and actual execution plans in graphical format and the graphical structure makes understanding most plans very easy. However, the detailed data for the plan is hidden behind ToolTips and Property sheets, making it somewhat more difficult to get to, other than in a one-operator-at-a-time approach.

### Retrieving cached plans

There is some confusion regarding the different types of plan and what they really mean. I've heard developers talk about estimated and actual plans as if they were two completely different plans. Hopefully this section will clear things up. The salient point is that the query optimizer produces the plan, and there is only one valid execution plan for a query, at any given time.

When troubleshooting a long-running query retrospectively, we'll often need to retrieve the cached plan for that query from the plan cache. As discussed earlier, once the optimizer selects a new plan for a query, it places it in the plan cache, and passes it on to the query execution engine for execution. Of course, the optimizer never executes any queries, it merely formulates the plan based on its knowledge of the underlying data structures and statistical knowledge of the data. Cached plans don't contain any runtime information, except for the row counts in interleaved plans. 

We can retrieve this cached plan manually, via the Dynamic Management Objects, or using a tool such as Extended Events. We'll cover techniques to automate capture of the cached plan later in the book (Chapter 10).

### Plans for ad hoc queries: estimated and actual plans

Most of the time in this book, however, we'll retrieve the execution plan simply by executing ad hoc queries within SSMS. At the point we submit the query, we have the option to request either the estimated plan or the actual plan.

If we request the estimated plan, we do not execute the query; we merely submit the query for inspection by the optimizer, in order to see the associated plan. If there exists in the plan cache a plan that exactly matches the submitted query text, then the optimizer simply returns that cached plan. If there is no match, the optimizer performs the optimization process and returns the new plan. However, because there is no intent to execute the query, the next two steps are skipped (i.e. placing the plan in the cache, if it's a new plan, and sending it for execution). Since estimated plans never access data, they are very useful during development for testing large, complex queries that could take a long time to run.

If, when we submit the query, we request a plan with runtime information, (what SSMS refers to as an actual plan), then all three steps in the process are performed. 

If there is a cached plan that exactly matches the submitted query text, then the optimizer simply passes the cached plan to the execution engine, which executes it, and adds the requested runtime values to the displayed plan. If there is no cached plan, the optimizer produces a new plan, places it in the cache and passes it on for execution and, again, we see the plan with runtime information. For example, we'll see runtime values for the number of rows returned and the number of executions of each operator, alongside the optimizer's estimated values. Note that SQL Server does not store anywhere a second copy of the plan with the runtime information. These values are simply injected into the copy of the plan, whether displayed in SSMS, or output through other means.

### Will the estimated and actual plans ever be different?

Essentially, the answer to this is "No." As emphasized previously, there is only one valid execution plan for a query at any given time, and the estimated and actual plans will not be different.

You may see differences in parallelization between the runtime plan and the estimated plan, but this doesn't mean the execution engine "changed" the plan. At compile time, if the optimizer calculates that the cost of the plan might exceed the cost threshold for parallelism, then it produces two versions of the plan (parallelized and non-parallelized) and the execution engine chooses the appropriate one at runtime.

Sometimes, you might generate an estimated plan and then, later, an actual plan for the same query, and see that the plans are different. In fact, what will have happened here is that, in the time between the two requests, something happened to invalidate the existing plan in the cache, forcing the optimizer to perform a full optimization and generate a new plan. For example, changes in the data or data structures might have caused SQL Server to recompile the plan. Alternatively, processes or objects within the query, such as interleaving data definition language (DDL) and data manipulation language (DML), result in a recompilation of the execution plan.

If you request an actual plan and then retrieve from the cache the plan for the query you just executed (we'll see how to do that in Chapter X), you'll see that the cached plan is the same as your actual plan, except that the actual plan has runtime information.

One case where the estimated and actual plans will be genuinely different is when the estimated plan won't work at all. For example, try generating an estimated plan for the simple bit of code in Listing 1-4.



In [0]:
--Listing 1-4
CREATE TABLE TempTable
    (
      Id INT IDENTITY(1, 1) ,
      Dsc NVARCHAR(50)
    );
INSERT  INTO TempTable
        ( Dsc 
        )
        SELECT  [Name]
        FROM    [Sales].[Store];
SELECT  *
FROM    TempTable;
DROP TABLE TempTable;

You will get this error:

> Msg 208, Level 16, State 1, Line 7
> Invalid object name 'TempTable'.

The optimizer runs the statements through the algebrizer, the process outlined earlier that is responsible for verifying the names of database objects but, since SQL Server has not yet executed the query, the temporary table does not yet exist. 

The plan will get marked for deferred name resolution. In other words, while the batch is parsed, bound, and compiled, the SELECT query is excluded from compilation because the algebrizer has marked it as deferred. Capturing the estimated plan doesn't execute the query, and so doesn't create the temporary table, and this is the cause of the error. At runtime, the query will be compiled and now a plan does exist. If you execute Listing 1-4 and request the actual execution plan, it will work perfectly.

A second case where the estimated and actual plans will be different, new in SQL Server 2017, is when the optimizer uses interleaved execution. If we request an estimated plan for a query that contains a multi-statement table-valued function (MSTVF), then the optimizer will use a fixed cardinality estimation of 100 rows for the MSTVF. However, if we request an actual plan, the optimizer will first generate the plan using this fixed estimate, and then run the subtree containing the MSTVF to get the actual row counts returned, and recompile the plan based on these real row counts. Of course, this plan will be stored in the plan cache, so subsequent requests for either an estimated or actual plan will return the same plan.

### Capturing graphical plans in SSMS

In SSMS, we can capture both the estimated and the actual plans for a query, and there are several ways to do it, in each case. Perhaps the most common, or at least the route I usually take, is to use the icons in the toolbar. Figure 1-2 shows the Display Estimated Execution Plan icon.

![Figure 1-2: Capturing the Estimated plan](Graphics\Figure1-2.png) 

A few icons to the right, we have the Include Actual Execution Plan icon, as shown in Figure 1-3.
 
![Figure 1-3: Capturing the actual plan.](Graphics\Figure1-3.png)

Alternatively, for either type of plan, you could:
* right-click in the query window and select the same option from the context menu
* click on the Query option in the menu bar and select the same choice
* use the keyboard shortcut (CTRL+L for estimated; CTRL+M for actual within SSMS or CTRL+ALT+L and CTRL+ALT+M for the same within Visual Studio).

For estimated plans, we have to click the icon, or use one of the alternative methods, each time we want to capture that type of plan for a query. For the actual plan, each of these methods acts as an "on/off" switch for the query window. When the actual plan is switched on, at each execution, SQL Server will then capture an actual execution plan for all queries run from that window, until you turn it off again for each query window within SSMS.

Finally, there is one additional way to view a graphical execution plan, a Live Execution Plan. The view of the plan is based on a DMV, sys.dm_exec_query_statistics_xml, introduced in SQL Server 2014. This DMV returns live statistics for the operations within an execution plan. The graphical view of this DMV was introduced in SQL Server 2016. You toggle it on or off similar to what you do with an Actual execution plan. Figure 1-4 shows the button:
 
![Figure 1-4: Enabling the Live execution plan](Graphics\Figure1-4.png)
We’ll explore this completely in Chapter X.

# Capturing our first plan

It's time to capture our first execution plan. We'll start off with a relatively simple query that nevertheless provides a fairly complete view into everything you're going to do when reading execution plans.

As noted in the introduction to this book, we strongly encourage you to follow along with the examples, by executing the relevant script and viewing the plans. Occasionally, especially as we reach more complex examples later in the book, you may see a plan that differs from the one presented in the book. This might be because we are using different versions of SQL Server (different service pack levels and cumulative updates), different editions, or we are using slightly different versions of the AdventureWorks sample database. We use AdventureWorks2016 in this book; other versions are slightly different, and even if you use the same version, its schema or statistics may have been altered over time. So, while most of the plans you get should be very similar, if not identical, to what we display here, don't be too surprised if you try the code and see something different.

Open a new query tab in SSMS and run the query shown in Listing 1-4.



In [0]:
--Listing 1-4
SELECT p.LastName + ', ' + p.FirstName,
    p.Title,
    pp.PhoneNumber
FROM Person.Person AS p
    INNER JOIN Person.PersonPhone AS pp
        ON pp.BusinessEntityID = p.BusinessEntityID
    INNER JOIN Person.PhoneNumberType AS pnt
        ON pnt.PhoneNumberTypeID = pp.PhoneNumberTypeID
WHERE pnt.Name = 'Cell'
      AND p.LastName = 'Dempsey';

Click the Display Estimated Execution Plan icon and in the execution plan tab you will see the estimated execution plan, as shown in Figure 1-5.
  
![Figure 1-5: Estimated execution plan](Graphics\Figure1-5.png)

Notice that there is no Results tab, because we have not actually executed the query. Now, highlight the Include Actual Execution Plan icon and execute the query. This time you'll see the result set retuned (a single row) and the Execution plan tab will display the actual execution plan, which should also look as shown in Figure 1-5.

# The components of a graphical execution plan

We're now going to explore each section of the plan from Figure 1-5 in more detail, but still at a high level. We won't start exploring the details of individual operators until Chapter 3. You'll notice that it's rather difficult to read the details on the plan in Figure 1-5. Here, and throughout the book we'll be following a method where I show the whole plan, and then drill down into sections of the plan to discuss individual parts or segments of the plan.

Most people start on the right-hand side, when reading plans, where you will find the operators that read data out of the base tables and indexes. From there we follow the data flow, as indicated by the arrows, from right to left until it reaches the SELECT operator, where the rows are passed back to the client. However, it's equally valid to read the plan from left-to-right, which is the order in which the operators are called – essentially data is pulled right-to-left as each operator in turn calls the child operator on its right, but we'll discuss this in more detail in Chapter 3.

## Operators

Operators, represented as icons in the plan, are the workhorses of the plan. Each operator implements a specific algorithm designed to perform a specialized task. The operators in a plan tell us exactly how SQL Server chose to execute a query, such as how it chose to access the data in a certain table, how it chose to join that data to rows in a second table, how and where it chose to perform any aggregations, sorting, calculations, and so on. 

In this example, let's start on the right-hand side of the plan, with the operators shown in Figure 1-6.
  
![Figure 1-6: Two data access operators and a join operator](Graphics\Figure1-6.png)

Here we see two data access operators passing data to a join operator. The first operator is an Index Seek, which is pulling data from the Person table using a nonclustered index, Person.IX_Person_LastName_FirstName_MiddleName. Each qualifying row (rows where the last name is Dempsey) passes to a Nested Loops operator, which is going to pull additional data, not held in the nonclustered index, from the Key Lookup operator.

Each operator has both a physical and a logical element. For example, in Figure 1-6, Nested Loops is the physical operator, and Inner Join is the logical operation it performs. So the logical component describes what the operator actually does (an INNER JOIN operation) and the physical part is how the optimizer chose to implement it (using a Nested Loops algorithm).

From the first Nested Loops operator, the data flows to a Compute Scalar operator. For each row, it performs its required task (in this case, concatenating the first and last names with a comma) and then passes it on to the operator on its left. This data is joined with matching rows in the PersonPhone table, and then in turn with matching rows in the PhoneNumberType table. Finally, the data flows to the SELECT operator.
  
![Figure 1-7: Broader section of the plan showing more operators](Graphics\Figure1-7.png)

The SELECT icon is one that you're going to frequently reference for the important data it contains. Of course, every operator contains important data (see the Operator Properties section, a little later), but what sets the SELECT operator apart is that it contains data about the plan as a whole, whereas other icons only expose information about the operator itself. 

## Data flow arrows

The arrows represent the direction of data flow between the operators, and the thickness of the arrow reflects the amount of data passed, a thicker arrow meaning more rows. Arrow thickness is another visual clue as to where performance issues may lie. For example, you might see a big thick arrow emerging from a data access operator, on the right side of the plan, but very thin arrows on the left, since your query ultimately returns only two rows. This is a sign that a lot of data was processed to produce those two output rows. That may be unavoidable for the functional requirements of the query, but equally it might be something you can avoid.

You can hover with the mouse pointer over these arrows and it will show the number of rows that it represents in a ToolTip that you can see in Figure 1-8. In an execution plan that contains runtime statistics (the actual plan), the thickness is determined by the actual, rather than estimated, number of rows.
  
![Figure 1-8: Tool tip for the data flow arrow](Graphics\Figure1-8.png)

## Estimated operator costs

Below each individual icon in a plan is displayed a number as a percentage. This number represents the estimated cost for that operator relative to the estimated cost of the plan as a whole. These numbers are best thought of as "cost units," based on the mathematical calculations of anticipated CPU and I/O within the optimizer. The estimated costs are useful as measures, but these costs don't represent real-world measures of actual CPU and I/O. There is generally a correlation between high estimated cost within the plan, and higher actual performance costs, but these are still just estimated values.

> __The origin of the estimated cost values__

> The story goes that the developer tasked with creating execution plans in SQL Server 7 used his workstation as the basis for these numbers, and they have never been updated. See Danny Ravid's blog at: http://preview.tinyurl.com/yawet2l3.

All operators will have an associated cost, and even an operator displaying 0% will actually have a small associated cost, which you can see in the operator's properties (which we'll discuss shortly).

If you compare the operator- and plan-costs side by side for the estimated and actual plan of the same query, you'll see that they are identical. Only the optimizer generates these cost values, which means that all costs in all plans are estimates, based on the optimizer's statistical knowledge of the data.

## Estimated total query cost relative to batch

At the top of every execution plan is displayed as much of the query string as will fit into the window, and a "cost (relative to the batch)" of 100%. 

![Figure 1-9](Graphics\Figure1-9.png)

Just as each query can have multiple operators, and each of those operators will have a cost relative to the query, you can also run multiple queries within a batch and get execution plans for them. Each plan will then have different costs. The estimated cost of the total query is divided by the estimated cost of all queries in a batch. Each operator within a plan displays its estimated costs relative to the plan it's a part of, not to the batch as a whole. 

Never lose sight of the fact that the costs you see, even in actual plans, are an estimated cost, not real, measured, performance metrics. If you focus your tuning efforts exclusively on the queries or operators with high estimated costs, and it turns out the cost estimations are incorrect, then you may be looking in the wrong area for the cause of performance issues.

## Operator properties

Right-click any icon within a graphical execution plan and select the Properties menu item to get a detailed list of information about that operation. Each operator performs a distinct task and therefore each operator will have a distinct set of property data. The vast majority of useful information to help you read and understand execution plans is contained in the Properties window for each operator. It's a good habit to get into when reading an execution plan to just leave the Properties window open and pinned to your SSMS window at all times. Sadly, due to the vagaries of the SSMS GUI, you may sometimes have to click two places to get the properties you want to properly display.

Figure 1-10 compares the Properties window for the same Index Seek operator at the top right of Figure 1-4, which performs a seek operation on a non-clustered index on the Person table. The left-hand pane is from the estimated plan, and the right-hand pane is for the actual plan.
 
![Figure 1-10: Comparing properties of the Index Seek operator for the estimated and actual plans](Graphics\Figure1-10.png)

As you can see, in the actual plan we see the actual as well as the estimated number of rows that passed through that operator, as well as the actual number of times the operator was executed. Here we see that the optimizer estimated 1.3333 rows and 2 were actually returned.

When comparing the properties of an operator, for the estimated and actual plans, look out for very big differences between the estimated and actual number of rows returned, such as an estimated row count of 100 and an actual row count of 100,000 (or vice versa). If a query that returns hundreds of thousands of rows uses a plan the optimizer devised for returning 10 rows, it is likely to be very inefficient, and you will need to investigate the possible cause. It might be that the row count has changed significantly since the plan was generated but statistics have not yet auto-updated, or it might be caused by problems with parameter sniffing, or by other issues. We'll return to this topic in detail in Chapter X.

I'm not going into detail here on all the properties and their meanings, but I'll mention briefly a few that you'll refer to quite frequently:

* Actual Number of Rows – the true number of rows returned according to runtime statistics. The availability of this value in actual plans is the biggest difference between these and cached plans (or estimated plans). Look out for big differences between this value and the estimated value.
* Defined Values – values introduced by this operator, such as the columns returned, or computed expressions from the query, or internal values introduced by the query processor.
* Estimated Number of Rows – calculated based on the statistics available to the optimizer for the table or index in question. These are useful for comparing to the Actual Number of Rows.
* Estimated Operator Cost – the estimated operator cost as a figure (as well as a percentage). This is an estimated cost even in actual plans.
* Object – the object accessed, such as the index being accessed by a scan or a seek operation.
* Output List – columns returned.
* Predicate – a "pushed down" search predicate.
* Table Cardinality – number of rows in the table.

You'll note that some of the properties, such as Object, have a triangle icon on their left, indicating that they can be expanded. Some of the longer property descriptions have an ellipsis at the end, which allows us to open a new window, making the longer text easier to read. Almost all properties, when you click on them, display a description at the bottom of the Property pane.

All these details are available to help us understand what's happening within the query in question. We can walk through the various operators, observing how the subtree cost accumulates, how the number of rows changes, and so on. With these details, we can identify queries that are estimated to use excessive amounts of CPU or tables that need more indexes, or identify other performance issues.

## ToolTips

Associated with each of the icons and the arrows is a pop-up window called a ToolTip, which you can access by hovering your mouse pointer over the icon or arrow. I already used one of these in Figure 1-7. Essentially, the tooltip for an operator is a cut-down version of the full properties window. It’s worth noting that the ToolTip and the Properties for given operators change as SQL Server itself changes. You may see differences in the ToolTips between one version of SQL Server and the next. Most of the examples in this book are from SQL Server 2016.

Figure 1-11 shows the ToolTip window for the SELECT operator for the estimated execution plan for the query in Listing 1-4.
 
![Figure 1-11: ToolTip for the SELECT operator](Graphics\Figure1-11.png)

The properties of the SELECT operator are often particularly interesting, since this provides information relating to the plan as a whole. For example, we see the following two property values (among others, several of which we'll review in detail in Chapter 3):

* Cached plan size – how much memory the plan generated by this query will take up in the plan cache. This is a useful number when investigating cache performance issues because you'll be able to see which plans are taking up more memory. 
* Degree of Parallelism – whether this plan was designed to use (or used) multiple processors. This plan uses a single processor as shown by the value of 1.

In Figure 1-11, we also see the Statement that represents the entire query that SQL Server is processing. You may not see the Statement if it's too long to fit into the ToolTip window. The same thing applies to other properties in other operators. This is yet another reason to focus on using the Properties window when working with execution plans.

The information available in the tooltips can be extremely limited. But, it's fairly quick to see the information available in the tooltips since all you have to do is hover your mouse to get the tips. To get a more consistent and more detailed view of information about the operations within an execution plan, you should use the full Properties window.

## Saving execution plans

We can save an execution plan from the graphical execution plan interface by right-clicking within the execution plan and selecting Save Execution Plan As. Way back in SQL Server 2005, we then had to change the filter to "*.*" and, when typing the name of the file we wanted to save, add .sqlplan as the extension. Thankfully, SQL Server 2008, and later, automatically selects the .sqlplan file type. 

What we are saving is simply an XML file. One of the benefits of extracting an XML plan and saving it as a separate file is that we can share it with others. For example, we can send the XML plan of a slow-running query to a DBA friend and ask them their opinion on how to rewrite the query. Once the friend receives the XML plan, he or she can open it up in Management Studio and review it as a graphical execution plan.

You can look at the underlying XML of a plan as well by right-clicking on the plan and selecting Show Execution Plan XML from the context menu. That will open the raw XML in another window where you can browse the XML manually if like. Alternatively, you can open the .sqlplpan file in Notepad. We'll explore the XML within execution plans in detail in Chapter X.

# Summary

In this chapter, we've described briefly the role of the query optimizer in producing the execution plan for a query, and how it selects the lowest-cost plan, based on its knowledge of the data structures and statistical knowledge of the data distribution. We also covered the plan cache, the importance of plan reuse, and how to promote this.

We explored the different execution plan formats, and then focused on graphical execution plans, how to read these plans, and the various components of these plans. We are going to spend a lot of time within the graphical plans when interpreting individual execution plans, so understanding the information available within the plans is important.

I also tried to clear up any confusion regarding what the terms "estimated plan" and "actual plan" really mean. I've even heard people talk about "estimated and actual plans" as if they were two completely different plans, or that the estimated plan might be somehow "inaccurate." Hopefully this chapter dispelled those misunderstandings.